In [1]:
import pickle
import dotenv

In [2]:
import tqdm
import time
import re

In [3]:
from mmw.parse import *

In [4]:
dotenv.load_dotenv()

True

In [5]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response['choices'][0]['text']

'\n\nThis is indeed a test'

In [6]:
questions_explanation = parse_question_explanations()

In [7]:
all_parsed_essays = parse_essays()

In [8]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [9]:
len(all_parsed_essays), len(filtered_marked_essays)

(77, 76)

In [10]:
#model = "text-davinci-003"
#model = "text-ada-001"
model = "gpt-3.5-turbo"

#models simple/ cheap/ fast to advanced:
#model = "ada"
#model = "babbage"
#model = "curie"
#model = "davinci"

#model = "ada:ft-mark-my-words-2023-05-01-11-44-26"

In [11]:
prompt_tpl = 'You are a caring teacher who wants to help their students succeed by understanding their mistakes and how to improve. Given is a student essay and a graded remark to a question. Grades can be numbers between 0 (lowest, worst) and 3 (highest, best). Give a possible reason and detailed explanation for the graded remark along with a reference from the Essay. Use direct form to answer the student. Start with a positive feedback.\n\n###\n\n'
prompt_tpl += 'Essay:\n{essay}\n\n###\n\n'
prompt_tpl += 'Question:\n{explanation}\n{question}\n\n###\n\n'
prompt_tpl += 'Grade: {remark}\n\n###\n\n'
prompt_tpl += 'Answer:'

In [12]:
def assemble_citation_prompts(all_parsed_essays, filtered_marked_essays, questions_explanation):

    all_prompts = []
    for i,essay_array in tqdm.tqdm(enumerate(all_parsed_essays)):

        essay_id = int(re.findall(r'\d+', essay_array[0])[-1])
        cur_row = filtered_marked_essays[filtered_marked_essays['essay id'] == essay_id]
        if len(cur_row) != 1:
            continue

        for j,k in questions_explanation.iterrows():
            question = k.question
            explanation = k.explanation
            grade = int(cur_row.iloc[0,j+1])
            prompt = prompt_tpl.format(essay=essay_array[3][:-1], #implicit newline
                                    question=question.strip(),
                                    explanation=explanation.strip(),
                                    remark=grade)

            all_prompts.append((i, j, prompt, grade))
    return all_prompts


In [13]:
all_prompts = assemble_citation_prompts(all_parsed_essays, filtered_marked_essays, questions_explanation)

77it [00:00, 486.82it/s]


In [14]:
all_answers = []
for prompt_array in tqdm.tqdm(all_prompts[:100]): #[:LIMIT_PROMPTS*len(questions_explanation)]):
    for i in range(3):
        try:
            i, j, prompt, annot = prompt_array
            all_splits = prompt.split('\n\n###\n\n')
            message = '\n\n###\n\n'.join(all_splits[1:])
            response = openai.ChatCompletion.create(model=model, messages=[
                {'role': 'system', 'content': all_splits[0]},
                {'role': 'user', 'content': message}],
                                                    temperature=0, max_tokens=250)
            all_answers.append((i, j, prompt, response, annot))
            
            time.sleep(.5)
            break
        except:
            print(f"exception, retrying {i}")
            time.sleep(2.)

 20%|██        | 20/100 [06:46<22:23, 16.79s/it]

exception, retrying 1


100%|██████████| 100/100 [29:51<00:00, 17.91s/it]


In [15]:
save_results('data/evaluations/run_8_chatgpt_comments', all_parsed_essays, all_questions, all_answers)

In [16]:
all_answers[0][3]['choices'][0]['message']['content']

'Great job on writing a narrative essay! Your opening sentence definitely grabs the reader\'s attention by creating a sense of mystery and intrigue. However, there are a few areas where you can improve to make it more compelling. Your use of grammar and spelling needs some work, as there are several errors throughout the essay. Additionally, try to avoid going off-topic and stay focused on the main point of your essay. Keep up the good work and keep practicing your writing skills! \n\nRegarding the graded remark, you received a grade of 1 for the opening sentence. While it does create a sense of mystery, there are some areas where it could be improved to make it more compelling. For example, you could try to use more descriptive language to paint a vivid picture in the reader\'s mind. Additionally, try to avoid using slang or informal language, as it can detract from the overall quality of your writing. Keep practicing and experimenting with different techniques to improve your writing